In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten,Conv2DTranspose,Conv2D
from keras.layers import BatchNormalization
from keras.layers.activation import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Define input image dimensions
#Large images take too much time and resources.
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [3]:
##########################################################################
#Given input of noise (latent) vector, the Generator produces an image.
def build_generator():
    noise_shape = 100 #1D array of size 100 (latent vector / noise)
    model = Sequential()
    model.add(Dense(7 * 7 * 256, input_dim=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 256)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(1, (7, 7), activation='tanh', padding='same'))
    return model

def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [4]:
def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3) 
    half_batch = int(batch_size / 2)


    for epoch in range(epochs):

        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

        noise = np.random.normal(0, 1, (batch_size, 100)) 
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size
        g_loss = combined.train_on_batch(noise, valid_y)
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        if epoch % save_interval == 0:
            save_imgs(epoch)

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [5]:
disc_optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.
gen_optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=disc_optimizer,
    metrics=['accuracy'])
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=gen_optimizer)
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)
discriminator.trainable = False  
valid = discriminator(img)  #Validity check on the generated image
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)
train(epochs=2000, batch_size=32, save_interval=100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

1/1 [==============================] - 0s 138ms/step
0 [D loss: 0.638945, acc.: 43.75%] [G loss: 0.694773]
1/1 [==============================] - 0s 28ms/step
1 [D loss: 0.398793, acc.: 50.00%] [G loss: 0.652384]
1/1 [==============================] - 0s 29ms/step
2 [D loss: 0.406708, acc.: 50.00%] [G loss: 0.610299]
1/1 [==============================] - 0s 28ms/step
3 [D loss: 0.431193, acc.: 50.00%] [G loss: 0.587706]
1/1 [==============================] - 0s 28ms/step
4 [D loss: 0.437647, acc.: 50.00%] [G loss: 0.603511]
1/1 [==============================] - 0s 30ms/step
5 [D loss: 0.411721, acc.: 50.00%] [G loss: 0.684414]
1/1 [==============================] - 0s 27ms/step
6 [D loss: 0.363601, acc.: 84.38%] [G loss: 0.817209]
1/1 [==============================] - 0s 30ms/step
7 [D loss: 0.358897, acc.: 100.00%] [G loss: 0.838982]
1/1 [==============================] - 0s 29ms/step
8 [D loss: 0.377880, acc.: 96.88%] [G loss: 0.798060]
1/1 [==============================] - 0s 26

In [6]:
# save the generator, discriminator and gan model
discriminator.trainable = True
discriminator.save("discriminator_model.h5")
discriminator.save("discriminator_model")

optimizer_weights = discriminator.optimizer.get_weights()
np.save("discriminator_optimizer.npy", optimizer_weights)

INFO:tensorflow:Assets written to: discriminator_model\assets


c:\Users\shrid\anaconda3\envs\deep_learning\Lib\site-packages\numpy\lib\npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [7]:
generator.save("generator_model.h5")
generator.save("generator_model")

gen_optimizer_weights = generator.optimizer.get_weights()
np.save("generator_optimizer.npy", gen_optimizer_weights)

INFO:tensorflow:Assets written to: generator_model\assets


INFO:tensorflow:Assets written to: generator_model\assets


In [8]:
discriminator.trainable = False
combined.save("gan_model.h5")
combined.save("gan_model")

gan_optimizer_weights = combined.optimizer.get_weights()
np.save("gan_optimizer.npy", gan_optimizer_weights)

INFO:tensorflow:Assets written to: gan_model\assets


INFO:tensorflow:Assets written to: gan_model\assets
c:\Users\shrid\anaconda3\envs\deep_learning\Lib\site-packages\numpy\lib\npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
